In [ ]:
%load_ext autoreload
%autoreload 2

#%pip install magic_duckdb --upgrade --pre --quiet
%load_ext magic_duckdb

%dql -t show pragma version

types = %dql --listtypes
print(types)

In [ ]:
# List available functions
%dql -d

%dql -t show pragma version 

In [ ]:
# Line magic with PyArrow
%dql -t arrow

%dql CREATE OR REPLACE TABLE xyz as SELECT * FROM range(100) t(x)

table1 = %dql SELECT * FROM xyz where x>50

display(table1)

In [ ]:
# List tables used by a query
%dql --tables with abc as (select * from def) select * from abc, tbl1, tbl3


In [ ]:
# Line magic with Pandas DataFrame
%dql -t df
%dql CREATE OR REPLACE TABLE xyz as SELECT * FROM range(100) t(x)

df1 = %dql SELECT * FROM xyz where x>50

display(len(df1))

In [ ]:
%%dql

CREATE OR REPLACE TABLE abc as SELECT * FROM range(200) c(a) where a % 7 == 0;
CREATE OR REPLACE TABLE def as SELECT * FROM range(200) c(a) where a % 2 == 0;

SELECT count(*) FROM def join abc on def.a=abc.a;

In [ ]:
# Get the connection created within DQL and use it directly
con = %dql --getcon
display(con.sql("pragma version").df())

In [ ]:
# Use an externally created connection
import duckdb

dql_test_con = duckdb.connect("testdbfile6.db")
dql_test_con.sql("CREATE OR REPLACE TABLE xyz as select * from range(100)")

# -co sets the connection to an existing object
%dql -co dql_test_con
%dql select count(*) from xyz

In [ ]:
# Pass a connection string via dql -cn
%dql -cn testdbfile6.db
%dql create or replace table xyz as select * from range(1);select count(*) from xyz

In [ ]:
# Pass a connection string via dql -cn, in this case, :memory: which is the same as using the -d default connection
%dql -cn :memory:
%dql pragma version

In [ ]:
# Combining DQL and a relationship

%dql -t relation
r = %dql select * from (select * from range(105) t(x)) join (select * from range(10)) on true
r.filter('x > 60').show()

In [ ]:
%%dql -t arrow
-- Cell Magic with a different output format Option
select * from (select * from range(105) t(x)) join (select * from range(10)) on true


In [ ]:
%dql -t show select * from (select * from range(105) t(x)) join (select * from range(10)) on true


In [ ]:
# Get the last variable. This is one way to get the output of a cell and line magic. Another way is: obj = %dql ... 
r = _
print(r)

In [ ]:
# Miscellaneous examples

# Set the default_connection
duckdb.default_connection = duckdb.connect(":memory:") 
# DQL will use the default connection. 
%dql -d  
# returns the dql connection object
con = %dql --getcon 
%dql -t df
# stores result of pragma version in object named myversion
%dql -o myversion pragma version 
display(myversion)

In [ ]:
# Jinja Template
# define a var, use -r to replace the variables inside the magic, then confirm it worked

table1 = "mytableisawesome"
%dql -t df --jinja2 create or replace table {{table1}} as select * from range(100)
%dql select * from mytableisawesome


In [ ]:
# List tables

table1 = "mytableisawesome"
%dql -t df --jinja2 create or replace table {{table1}} as select * from range(100)
%dql select * from mytableisawesome

In [ ]:
# Pass parameter

obj1 = [1,2,3]
r1 = %dql -p obj1 select ?
display(r1)

obj1 = [1,2,3]
r2 = %dql -p obj1 select 2 in (select unnest(?))
display(r2)

# Passing two parameters
obj1 = [1,2,3]
obj2 = "b"
r3 = %dql -p obj1 -p obj2 select 2 in (select unnest(?)), 'abc' = ?
display(r3)

# Passing two parameters plus jinja template
obj1 = [1,2,3]
obj2 = "b"
obj3 = "this will be passed through jinja2 template"
r4 = %dql -p obj1 -p obj2 --jinja2 select 2 in (select unnest(?)), 'abc' = ?, '{{obj3}}'
display(r4)


In [ ]:
# Demonstrate connection configuration 

print('Connect via duckdb.default_connection() and run this query')
q1 = %dql -d PRAGMA database_list
display(q1)

print('Connect via duckdb.connect("somedb.db") and run this query')
q2 = %dql -cn somedb.db PRAGMA database_list
display(q2)

print('Run this query with whatever the global magic_duckdb connection is, which defaults to duckdb.default_connection()')
q3 = %dql PRAGMA database_list
display(q3)

print('Connect via duckdb.connect("somedb2.db") and set as global magic_duckdb.connection')
q4 = %dql -cn somedb2.db

print('Run a query, using the global magic_duckdb.connection')
q5 = %dql PRAGMA database_list
display(q5)
